This notebook tries to add new features into the model, including the acid proportions of the whole sequence.

In [1]:
!pip install Levenshtein
import pandas as pd
import numpy as np
import Levenshtein

# First read the grouped_data
grouped_data = pd.read_csv("grouped_sample_no_pH_one_muta.csv")
grouped_data = grouped_data.iloc[np.array(list(grouped_data.group))!=-1,:]
group_num = grouped_data.group.nunique()
print(group_num)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 32.9 MB/s 
     |████████████████████████████████| 2.2 MB 48.9 MB/s 
94


In [2]:
# Check what is the maximum number of mutations in a same group
import Levenshtein

group_max_nmuta = []
for group_id in range(group_num):
    group_nmuta = []
    group = grouped_data.iloc[np.array(list(grouped_data.group))==group_id,1]
    for ind1 in range(len(group)-1):
        seq1 = group.iloc[ind1]
        for ind2 in range(ind1+1,len(group)):
            seq2 = group.iloc[ind2]
            group_nmuta.append(Levenshtein.distance(seq1, seq2))
    group_max_nmuta.append(max(group_nmuta))
print(group_max_nmuta)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2]


In [3]:
# This block presents the functions to extract features from a pair of protein sequences
def acid_proportion(neighbor):
    '''
    To calculate the proportion of acids in the protein acid sequence called "neighbor". 

    Parameters
    ----------
    neighbor: a string in the shape of "AGHEF...".
        A sub-sequence of the protein acid sequence. 

    Returns
    ----------
    The return is in the shape of [0.02, 0.12, ..., 0.23], a list of length = 20. The numbers 
        in the list correspond to acids A, B, C, ..., T respetively, representing their proportions
        in the protein acid sequence called "neighbor". So these 20 numbers should sum up to 20.
    '''
    neighbor_len = len(neighbor)
    alpha_list = "A B C D E F G H I J K L M N O P Q R S T".split()
    results = [neighbor.count(alpha)/neighbor_len for alpha in alpha_list]
    return(results)


def pair_mutation_features(seq1, seq2, neibor_radius=15, new_feature=False):
    '''
    To extract the featurs given a pair of similar protein sequences with only one mutation.
    Features include the mutation type, and also the proportions of acids in the neighbor of 
        the muation position in seq1.

    Parameters
    ----------
    seq1, seq2: strings in the shape of "AABSD...".
        The sequences indicate the real acid composition of proteins.
        Note that seq1 must be not shorter than seq2, and seq1 must be different from seq2.
        Also, there should be only one mutation in this pair.

    neibor_radius: Integer, larger than 0.
        This number decides the radius of the neighbor of the mutation
        spot. The acid proportion will be calculated within that neighbor.

    new_feature: Boolean type.
        Indicating wether or not to add the acid proportions in the whole sequence as a new feature

    Returns:
    ----------
    The output is in the shape of ["A-to-B", 0.1, 0.03, ..., 0.04].
    The first item is a categorical str variable, which comes from 400 possible choices.
    The remaining a list showing the proportions of acids in the neighbor of mutation.
    '''
    # First find the position and category of the mutation
    for acid_ind in range(len(seq1)):
        if (acid_ind > len(seq2)-1):
            category = seq1[acid_ind] + "-to- "
            mutation_pos = acid_ind
            break
        elif (seq1[acid_ind] != seq2[acid_ind]) :           # At this point the mutation position is found
            if seq1[acid_ind+1:] == seq2[acid_ind:]:     # This means the mutation is deletion
                category = seq1[acid_ind] + "-to- "
            elif seq1[acid_ind+1:] == seq2[acid_ind+1:]:       # This mean the mutation is substitution
                category = seq1[acid_ind] + "-to-" + seq2[acid_ind]
            else:
                return("Error happens for this pair")
            mutation_pos = acid_ind
            break
    
    # Now tailor the neighbor in case it exceeds the bounds
    mutation_neibor = [mutation_pos-neibor_radius, mutation_pos+neibor_radius]
    if mutation_neibor[0] < 0:
        mutation_neibor = [0, 2*neibor_radius]
    elif mutation_neibor[1] > len(seq1) - 1:
        mutation_neibor = [len(seq1)-1-2*neibor_radius, len(seq1)-1]
    mutation_neibor = [max(mutation_neibor[0],0), min(len(seq1)-1, mutation_neibor[1])]

    # Now do the acid proportion calculation on the neighbor
    neighbor = seq1[mutation_neibor[0]:mutation_neibor[1]]
    acids_proportion = acid_proportion(neighbor)

    # Also, add the acid proportions of the whole sequence as a new feature
    if new_feature == True:
        acids_proportion += acid_proportion(seq1)
    return([category] + acids_proportion)


In [4]:
# This block presents functions to judge whether there are more than one mutations
# in a pair of sequences. If so, divide this pair of sequences into more pairs
# that correspond to every mutation respectively.
import Levenshtein

def seperate_pairs(seq1, seq2):
    '''
    Judge whether there are more than one mutations happening in the sequence pair.
    If yes, generate new pairs that correspond to each mutation respectively.

    Parameters
    ----------
    seq1, seq2: strings in the shape of "AABSD...".
        The sequences indicate the real acid composition of proteins.
        Note that seq1 must be not shorter than seq2, and seq1 must be different from seq2.
    
    Returns:
    ----------
    The return will be in the shape of [(seq1, seq3), (seq1, seq4)].
    The "seq3", "seq4" represent one of the mutation from seq1 to seq2 individually.
    I.e., the length of this list will be the same as the number of mutations
    in the original pair (seq1, seq2).

    Note:
    ----------
    Notice that the maximum number of mutations in one pair of sequences will not exceed 2,
    the function make use of this fact to decide the pair seperation strategy.
    '''
    num_mutations = Levenshtein.distance(seq1, seq2)
    if num_mutations == 1:
        return([(seq1, seq2)])
    # found_mutations = 0
    output_list = []
    for ind in range(len(seq1)):
        new_seq11 = seq1[:ind] + seq1[ind+1:]
        if Levenshtein.distance(new_seq11, seq2) < Levenshtein.distance(seq1, seq2):
            output_list.append((seq1, new_seq11))
            if len(output_list) == num_mutations:
                break
            seq2 = seq2[:ind] + seq1[ind] + seq2[ind:]
            
        elif ind < len(seq2): 
            new_seq12 = seq1[:ind] + seq2[ind] + seq1[ind+1:]
            if Levenshtein.distance(new_seq12, seq2) < Levenshtein.distance(seq1, seq2):
                output_list.append((seq1, new_seq12))
                if len(output_list) == num_mutations:
                    break
                seq2 = seq2[:ind] + seq1[ind] + seq2[ind+1:]
    return(output_list)

# Test
print(seperate_pairs("abssbwbsbwaad", "absbbwbsbwaac"))
        

[('abssbwbsbwaad', 'absbbwbsbwaad'), ('abssbwbsbwaad', 'abssbwbsbwaac')]


In [5]:
# These are the functions to generate the prediction target, y
def get_y1(temp1, temp2, num_mutations):
    y = temp2 - temp1 
    y /= num_mutations
    return(y)

def get_y2(temp1, temp2, num_mutations):
    y = temp2 / temp1
    y = num_mutations**(1/num_mutations)
    return(y)

# This is the function to calculate whether the difference between two temperatures is large enough to be considered,
# as it is necessary to reduce the training dataset size (the RAM could not afford the load)
def temp_diff_measure(temp1, temp2):
    return(abs(temp1-temp2)/temp1)

# This is the function to get the target features of sequences in the same group。
def get_group_features(group, neibor_radius=15, new_feature=False):

    group_features = []
    group_nrow = group.shape[0]
    for row_ind1 in range(group_nrow-1):
        current_seq = group.iloc[row_ind1,:]["protein_sequence"]
        current_tm = group.iloc[row_ind1,:]["tm"]
        for row_ind2 in range(row_ind1+1, group_nrow):
            pair_seq = group.iloc[row_ind2,:]["protein_sequence"]
            pair_tm = group.iloc[row_ind2,:]["tm"]
            difference_measure = temp_diff_measure(current_tm, pair_tm)
            if difference_measure < 0.2:            #!!!!!! This is to reduce the training dataset size, leaving out less obvious samples
                continue
            if current_seq != pair_seq:             # The pair is effective only when the sequences are different
                if len(current_seq) >= len(pair_seq):
                    re_pair = seperate_pairs(current_seq, pair_seq)
                    num_mutations = len(re_pair)
                    for pair in re_pair:
                        seq1 = pair[0]
                        seq2 = pair[1]
                        # This is for the case that y is the difference between
                        # Here if there are two mutations, the tm difference should be dilute into half for each pair
                        y = get_y2(current_tm, pair_tm, num_mutations)
                        group_features.append(pair_mutation_features(seq1,seq2,neibor_radius, new_feature=new_feature)+[y])

                else:
                    re_pair = seperate_pairs(pair_seq, current_seq)
                    num_mutations = len(re_pair)
                    for pair in re_pair:
                        seq1 = pair[0]
                        seq2 = pair[1]
                        y = get_y2(pair_tm, current_tm, num_mutations)
                        group_features.append(pair_mutation_features(seq1,seq2,neibor_radius, new_feature=new_feature)+[y])

    return(group_features)

In [ ]:
# This block extract the features in all groups
training_features = []
neibor_radius = 25          # !!!!!! This sets the radius of the neighborhood of the mutation position, in preparation for the calculation of acid proportions

new_feature=True        # This means that we try the new feature, acid proportions in the whole amino acid sequence
for group_ind in range(grouped_data.group.nunique()):
    # if group_ind == 125:            # As number of mutations in the seq pairs of group 125 exceed 2
    #     continue
    print(group_ind, "/", grouped_data.group.nunique())
    group = grouped_data.iloc[np.array(list(grouped_data.group))==group_ind,:]
    group_features = get_group_features(group,neibor_radius,new_feature=new_feature)
    training_features += group_features
train = pd.DataFrame(training_features)
print(train.shape)


In [7]:
col_names = ["mutation_category"] + ["acid_proportion"+str(i) for i in range(20)] + ["tem_diff"]
if new_feature:
    col_names = ["mutation_category"] + [str(i) for i in range(40)] + ["tem_diff"]
train.columns = col_names

In [8]:
# This block sets up the encoder for the mutation category
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
enc = OneHotEncoder()          # Initialization

# The one-hot encoder only consider the mutations in the training set,
# and later when comparing two sequences in the test set, if new mutations
# appear, just assign 0 to the code array
enc.fit(pd.DataFrame(np.reshape(train["mutation_category"].unique(), [len(train["mutation_category"].unique()),1])))   
train_coded_cate = enc.transform(train[["mutation_category"]]).toarray()

train_coded_cate = pd.DataFrame(train_coded_cate)
train_coded_cate.columns = ["category "+str(i) for i in range(train_coded_cate.shape[1])]
available_categories = train["mutation_category"].unique()         # This is the coverage of possible mutation categories in the training set

encoded_train = pd.concat([train_coded_cate, train.iloc[:,1:]], axis=1)
print(encoded_train.shape)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


(342418, 436)


In [9]:
# This block tries to construct the training model
train_x = encoded_train.iloc[:,:encoded_train.shape[1]-1]              # Leave out the last neighbor_acid_proportion as it is redundant
train_y = np.array([[i] for i in encoded_train["tem_diff"]])

import xgboost as xgb
from sklearn.model_selection import train_test_split
tr_x, va_x, tr_y, va_y = train_test_split(train_x, train_y, test_size=0.1, random_state=123)

xgb_parms = { 
    'max_depth':4, 
    'learning_rate':0.005, 
    'subsample':0.6,
    'colsample_bytree':0.2, 
    'eval_metric':'rmse',
    'objective':'reg:squarederror',
    'random_state':123
}
model = xgb
dtrain = xgb.DMatrix(data=tr_x, label=tr_y)
dvalid = xgb.DMatrix(data=va_x, label=va_y)


In [10]:
# This block generate the training model
model1 = xgb.train(xgb_parms, dtrain=dtrain,evals=[(dtrain,"train"),(dvalid, "valid")],
                 num_boost_round=2100,
                 early_stopping_rounds = 100,
                 verbose_eval = 100)
model1.save_model('demo1.model')

[0]	train-rmse:0.906472	valid-rmse:0.90594
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
[100]	train-rmse:0.549668	valid-rmse:0.549832
[200]	train-rmse:0.334663	valid-rmse:0.334662
[300]	train-rmse:0.205465	valid-rmse:0.205415
[400]	train-rmse:0.128608	valid-rmse:0.128745
[500]	train-rmse:0.084273	valid-rmse:0.084683
[600]	train-rmse:0.060325	valid-rmse:0.060959
[700]	train-rmse:0.048542	valid-rmse:0.049435
[800]	train-rmse:0.043328	valid-rmse:0.044404
[900]	train-rmse:0.041173	valid-rmse:0.042334
[1000]	train-rmse:0.040287	valid-rmse:0.041491
[1100]	train-rmse:0.039853	valid-rmse:0.041117
[1200]	train-rmse:0.039631	valid-rmse:0.040919
[1300]	train-rmse:0.039497	valid-rmse:0.040777
[1400]	train-rmse:0.039385	valid-rmse:0.040669
[1500]	train-rmse:0.039248	valid-rmse:0.040577
[1600]	train-rmse:0.039152	valid-rmse:0.040493
[1700]	train-rmse:0.039051	valid-rmse:0.040404
[1800]	train-rmse:0.0

In [11]:
# Now look at the test set.
# The strategy to rank the samples in the test set is to find the 
# wildtype in the test set first, then compare it with every sample
# in the test set, and finally sort the samples by their regression 
# result.

# With ZYZ's code, found that the 1170-th sample in the test data is the wildtype for the test set 
test_data = pd.read_csv("test.csv")
seq = list(test_data.protein_sequence)

prediction = []
test_wt = seq[1169]
for sample in seq:
    if sample == test_wt:
        prediction.append(0)
        continue
    else:
        mutation_features = pair_mutation_features(test_wt, sample, neibor_radius=neibor_radius, new_feature=True)
        if mutation_features[0] not in available_categories:
            mutation_features = [0] * train_coded_cate.shape[1] + mutation_features[1:]
        else:

            coded_cate = enc.transform([[mutation_features[0]]]).toarray().tolist()[0]
            mutation_features = coded_cate + mutation_features[1:] 

        mutation_features = pd.DataFrame([mutation_features])
        mutation_features.columns = ["category "+str(i) for i in range(train_coded_cate.shape[1])] + [str(i) for i in range(40)]  
        mutation_features = xgb.DMatrix(data=mutation_features)      
        prediction.append(model1.predict(mutation_features))


In [12]:
# Finally, rank all the test samples by the predictions we got above
results = pd.DataFrame()
results["seq_id"] = list(test_data.seq_id)
results["tm"] = [float(i) for i in prediction]
results.to_csv("results.csv",index=0)
# Upload to Kaggle